In [1]:
import pandas as pd
import yfinance as yf
import pandas_datareader.data as pdr
import datetime as dt
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('spy_v2.csv')
df

,period,movement,chg,vpoc,total_volume,vpoc_tpoc_ratio,vpoc_close_ratio,tpoc_close_ratio,vpoc_price_hl_range,tpoc_price_hl_range,price_cv,price_skew,price_kurt,vol_price_cv,vol_price_skew,vol_price_kurt,trapped_longs,trapped_shorts,trapped_shorts_at_extremes,trapped_longs_at_extremes
0,2024-03-26 09:30:00,1,0.0249,102621.0,277252.0,0.164300,0.263692,0.099391,0.417850,0.253550,0.0,-0.132,-1.380,2.791,7.165,53.192,0,1,0,0
1,2024-03-26 09:35:00,0,-0.0485,6333.0,108210.0,0.424460,0.458034,0.033573,0.299760,0.724221,0.0,0.928,1.374,0.726,1.262,1.483,1,0,0,0
2,2024-03-26 09:40:00,1,0.0211,8650.0,135346.0,0.000000,0.981651,0.981651,1.000000,1.000000,0.0,0.260,-1.013,0.696,1.534,2.873,0,1,1,0
3,2024-03-26 09:45:00,1,0.0025,8100.0,61379.0,0.170213,0.287234,0.457447,0.654255,0.824468,0.0,1.069,0.779,1.019,3.274,14.367,0,1,1,0
4,2024-03-26 09:50:00,0,-0.0447,4300.0,41548.0,0.765396,0.736070,0.029326,0.058651,0.824047,0.0,-0.746,-0.571,0.920,2.532,8.431,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,2024-05-02 15:35:00,0,-0.0135,9553.0,125001.0,0.408578,0.094808,0.503386,0.460497,0.869074,0.0,-0.362,-1.243,0.858,2.089,5.270,0,1,1,0
2024,2024-05-02 15:40:00,0,-0.0785,8071.0,91749.0,0.080392,0.127451,0.047059,0.749020,0.829412,0.0,0.423,-1.150,0.788,2.599,9.654,1,0,0,0
2025,2024-05-02 15:45:00,1,0.0700,11903.0,161357.0,0.046358,0.110927,0.064570,0.076159,0.122517,0.0,-1.140,0.867,0.842,1.791,3.317,1,0,0,1
2026,2024-05-02 15:50:00,0,-0.0353,16776.0,227468.0,0.583333,0.644928,0.061594,0.108696,0.692029,0.0,-0.350,-0.550,0.802,2.164,5.165,1,0,0,0


In [3]:
df.iloc[:,3:] = df.iloc[:,3:].shift(1,fill_value=-1)
df = df.rename(columns={c: c+"_prev_period" for c in df.columns if c not in ['period','movement','chg']})
df.insert(2,'movement_prev_period',df.movement.shift(1, fill_value=-1))
df.insert(4,'chg_prev_period',df.chg.shift(1))
df

,period,movement,movement_prev_period,chg,chg_prev_period,vpoc_prev_period,total_volume_prev_period,vpoc_tpoc_ratio_prev_period,vpoc_close_ratio_prev_period,tpoc_close_ratio_prev_period,...,price_cv_prev_period,price_skew_prev_period,price_kurt_prev_period,vol_price_cv_prev_period,vol_price_skew_prev_period,vol_price_kurt_prev_period,trapped_longs_prev_period,trapped_shorts_prev_period,trapped_shorts_at_extremes_prev_period,trapped_longs_at_extremes_prev_period
0,2024-03-26 09:30:00,1,-1,0.0249,NaN,-1.0,-1.0,-1.000000,-1.000000,-1.000000,...,-1.0,-1.000,-1.000,-1.000,-1.000,-1.000,-1,-1,-1,-1
1,2024-03-26 09:35:00,0,1,-0.0485,0.0249,102621.0,277252.0,0.164300,0.263692,0.099391,...,0.0,-0.132,-1.380,2.791,7.165,53.192,0,1,0,0
2,2024-03-26 09:40:00,1,0,0.0211,-0.0485,6333.0,108210.0,0.424460,0.458034,0.033573,...,0.0,0.928,1.374,0.726,1.262,1.483,1,0,0,0
3,2024-03-26 09:45:00,1,1,0.0025,0.0211,8650.0,135346.0,0.000000,0.981651,0.981651,...,0.0,0.260,-1.013,0.696,1.534,2.873,0,1,1,0
4,2024-03-26 09:50:00,0,1,-0.0447,0.0025,8100.0,61379.0,0.170213,0.287234,0.457447,...,0.0,1.069,0.779,1.019,3.274,14.367,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,2024-05-02 15:35:00,0,0,-0.0135,-0.1059,13200.0,164052.0,0.347445,0.132847,0.480292,...,0.0,-0.136,-1.162,0.813,2.543,7.962,1,0,0,0
2024,2024-05-02 15:40:00,0,0,-0.0785,-0.0135,9553.0,125001.0,0.408578,0.094808,0.503386,...,0.0,-0.362,-1.243,0.858,2.089,5.270,0,1,1,0
2025,2024-05-02 15:45:00,1,0,0.0700,-0.0785,8071.0,91749.0,0.080392,0.127451,0.047059,...,0.0,0.423,-1.150,0.788,2.599,9.654,1,0,0,0
2026,2024-05-02 15:50:00,0,1,-0.0353,0.0700,11903.0,161357.0,0.046358,0.110927,0.064570,...,0.0,-1.140,0.867,0.842,1.791,3.317,1,0,0,1


In [4]:
df = df[~df['period'].str.contains('09:30:00')].dropna().set_index('period')
df.index = pd.to_datetime(df.index)
df

,movement,movement_prev_period,chg,chg_prev_period,vpoc_prev_period,total_volume_prev_period,vpoc_tpoc_ratio_prev_period,vpoc_close_ratio_prev_period,tpoc_close_ratio_prev_period,vpoc_price_hl_range_prev_period,...,price_cv_prev_period,price_skew_prev_period,price_kurt_prev_period,vol_price_cv_prev_period,vol_price_skew_prev_period,vol_price_kurt_prev_period,trapped_longs_prev_period,trapped_shorts_prev_period,trapped_shorts_at_extremes_prev_period,trapped_longs_at_extremes_prev_period
period,,,,,,,,,,,,,,,,,,,,,
2024-03-26 09:35:00,0,1,-0.0485,0.0249,102621.0,277252.0,0.164300,0.263692,0.099391,0.417850,...,0.0,-0.132,-1.380,2.791,7.165,53.192,0,1,0,0
2024-03-26 09:40:00,1,0,0.0211,-0.0485,6333.0,108210.0,0.424460,0.458034,0.033573,0.299760,...,0.0,0.928,1.374,0.726,1.262,1.483,1,0,0,0
2024-03-26 09:45:00,1,1,0.0025,0.0211,8650.0,135346.0,0.000000,0.981651,0.981651,1.000000,...,0.0,0.260,-1.013,0.696,1.534,2.873,0,1,1,0
2024-03-26 09:50:00,0,1,-0.0447,0.0025,8100.0,61379.0,0.170213,0.287234,0.457447,0.654255,...,0.0,1.069,0.779,1.019,3.274,14.367,0,1,1,0
2024-03-26 09:55:00,1,0,0.0587,-0.0447,4300.0,41548.0,0.765396,0.736070,0.029326,0.058651,...,0.0,-0.746,-0.571,0.920,2.532,8.431,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-02 15:35:00,0,0,-0.0135,-0.1059,13200.0,164052.0,0.347445,0.132847,0.480292,0.867153,...,0.0,-0.136,-1.162,0.813,2.543,7.962,1,0,0,0
2024-05-02 15:40:00,0,0,-0.0785,-0.0135,9553.0,125001.0,0.408578,0.094808,0.503386,0.460497,...,0.0,-0.362,-1.243,0.858,2.089,5.270,0,1,1,0
2024-05-02 15:45:00,1,0,0.0700,-0.0785,8071.0,91749.0,0.080392,0.127451,0.047059,0.749020,...,0.0,0.423,-1.150,0.788,2.599,9.654,1,0,0,0


In [5]:
yf.pdr_override()

data = pdr.get_data_yahoo('SPY', start=str(df.index[0].date()) ,end=str(df.index[-1].date() + dt.timedelta(days=1)), interval='5m')
data.index = pd.to_datetime(data.index)
data = data.tz_localize(None)
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-03-26 09:30:00,521.229980,521.440002,520.880005,521.359985,521.359985,1571734
2024-03-26 09:35:00,521.349976,521.429993,520.969971,521.090027,521.090027,551075
2024-03-26 09:40:00,521.090027,521.208374,520.830017,521.155029,521.155029,1697398
2024-03-26 09:45:00,521.164978,521.280029,521.065979,521.190002,521.190002,411373
2024-03-26 09:50:00,521.190002,521.250000,520.890015,520.955017,520.955017,350673
...,...,...,...,...,...,...
2024-05-02 15:35:00,504.649994,504.739990,504.154999,504.500000,504.500000,1150678
2024-05-02 15:40:00,504.519989,504.649994,504.070007,504.130005,504.130005,848233
2024-05-02 15:45:00,504.130005,504.510010,503.859985,504.410004,504.410004,1321070


In [6]:
df = pd.merge(data, df, left_index=True, right_index=True)
df.index.name = 'period'

In [7]:
encoder = LabelEncoder()
df['period_idx'] = encoder.fit_transform(df.index.time)

In [8]:
df

,Open,High,Low,Close,Adj Close,Volume,movement,movement_prev_period,chg,chg_prev_period,...,price_skew_prev_period,price_kurt_prev_period,vol_price_cv_prev_period,vol_price_skew_prev_period,vol_price_kurt_prev_period,trapped_longs_prev_period,trapped_shorts_prev_period,trapped_shorts_at_extremes_prev_period,trapped_longs_at_extremes_prev_period,period_idx
period,,,,,,,,,,,,,,,,,,,,,
2024-03-26 09:35:00,521.349976,521.429993,520.969971,521.090027,521.090027,551075,0,1,-0.0485,0.0249,...,-0.132,-1.380,2.791,7.165,53.192,0,1,0,0,0
2024-03-26 09:40:00,521.090027,521.208374,520.830017,521.155029,521.155029,1697398,1,0,0.0211,-0.0485,...,0.928,1.374,0.726,1.262,1.483,1,0,0,0,1
2024-03-26 09:45:00,521.164978,521.280029,521.065979,521.190002,521.190002,411373,1,1,0.0025,0.0211,...,0.260,-1.013,0.696,1.534,2.873,0,1,1,0,2
2024-03-26 09:50:00,521.190002,521.250000,520.890015,520.955017,520.955017,350673,0,1,-0.0447,0.0025,...,1.069,0.779,1.019,3.274,14.367,0,1,1,0,3
2024-03-26 09:55:00,521.039978,521.270020,520.919983,521.250000,521.250000,446260,1,0,0.0587,-0.0447,...,-0.746,-0.571,0.920,2.532,8.431,1,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-02 15:35:00,504.649994,504.739990,504.154999,504.500000,504.500000,1150678,0,0,-0.0135,-0.1059,...,-0.136,-1.162,0.813,2.543,7.962,1,0,0,0,72
2024-05-02 15:40:00,504.519989,504.649994,504.070007,504.130005,504.130005,848233,0,0,-0.0785,-0.0135,...,-0.362,-1.243,0.858,2.089,5.270,0,1,1,0,73
2024-05-02 15:45:00,504.130005,504.510010,503.859985,504.410004,504.410004,1321070,1,0,0.0700,-0.0785,...,0.423,-1.150,0.788,2.599,9.654,1,0,0,0,74


In [24]:
df.index[73], df.index[73]+dt.timedelta(minutes=30)
df.loc[df.index[73]:df.index[73]+dt.timedelta(minutes=30)]

,Open,High,Low,Close,Adj Close,Volume,movement,movement_prev_period,chg,chg_prev_period,...,vol_price_kurt_prev_period,trapped_longs_prev_period,trapped_shorts_prev_period,trapped_shorts_at_extremes_prev_period,trapped_longs_at_extremes_prev_period,period_idx,success,relative_volume_to_930,relative_volume_prev_period,relative_vpoc_prev_period
period,,,,,,,,,,,,,,,,,,,,,
2024-03-26 15:40:00,519.635010,519.969971,519.169983,519.190002,519.190002,1572947,0,0,-0.0835,-0.0239,...,-0.607,1,0,0,0,73,0,1.055076,1.015007,0.567471
2024-03-26 15:45:00,519.190002,519.440002,519.025024,519.111389,519.111389,2910887,0,0,-0.0247,-0.0835,...,12.420,1,0,0,1,74,1,1.159021,1.098519,2.609653
2024-03-26 15:50:00,519.119995,519.309692,518.979980,519.140015,519.140015,3080175,1,0,0.0179,-0.0247,...,1.148,1,0,0,0,75,0,1.411294,1.217660,0.658450
2024-03-26 15:55:00,519.150024,519.200012,518.400024,518.669983,518.669983,9707933,0,1,-0.0821,0.0179,...,1.052,0,1,0,0,76,0,2.507408,1.776674,2.028088


In [9]:
MAX_TRADE_DURATION = 30 #mins
TARGET_PRICE_MOVEMENT = 0.3 #dollars
MAX_DRAWDOWN = TARGET_PRICE_MOVEMENT #dollars

df['success'] = 0

for period in df.index:
    entry_zone = df.loc[period]
    entry_zone_price = entry_zone.Open
    trading_zone = df.loc[period:period+dt.timedelta(minutes=MAX_TRADE_DURATION)]
    
    if entry_zone.trapped_shorts_at_extremes_prev_period:
        if trading_zone.High.max() >= entry_zone_price + TARGET_PRICE_MOVEMENT:
            df.loc[period,'success'] = 1

    elif entry_zone.trapped_longs_at_extremes_prev_period:
        if trading_zone.Low.min() <= entry_zone_price - TARGET_PRICE_MOVEMENT:
            df.loc[period,'success'] = 1


print(df[df.trapped_longs_at_extremes_prev_period==1].success.value_counts() / df[df.trapped_longs_at_extremes_prev_period==1].success.value_counts().sum(), df[df.trapped_shorts_at_extremes_prev_period==1].success.value_counts() / df[df.trapped_shorts_at_extremes_prev_period==1].success.value_counts().sum())

df[df.trapped_longs_at_extremes_prev_period==1].success.value_counts(), df[df.trapped_shorts_at_extremes_prev_period==1].success.value_counts()

success
1    0.665
0    0.335
Name: count, dtype: float64 success
1    0.67688
0    0.32312
Name: count, dtype: float64


(success
 1    133
 0     67
 Name: count, dtype: int64,
 success
 1    243
 0    116
 Name: count, dtype: int64)

In [10]:
days = df.index.date

for day in days:
    volume_at_930 = df.loc[str(day) + " 09:35:00", 'total_volume_prev_period']
    vpoc_at_930 = df.loc[str(day) + " 09:35:00", 'vpoc_prev_period']

    # relative_volume_to_930
    df.loc[str(day):str(day) + " ", 'relative_volume_to_930'] = df.loc[str(day):str(day) + " ", 'total_volume_prev_period'] / volume_at_930

    # relative_volume_to_prev
    df.loc[str(day):str(day) + " ", 'relative_volume_prev_period'] = df.loc[str(day):str(day) + " ", 'total_volume_prev_period'] / df.loc[str(day):str(day) + " ", 'total_volume_prev_period'].shift(1,fill_value=volume_at_930)

    # relative_vpoc_prev_period
    df.loc[str(day):str(day) + " ", 'relative_vpoc_prev_period'] = df.loc[str(day):str(day) + " ", 'vpoc_prev_period'] / df.loc[str(day):str(day) + " ", 'vpoc_prev_period'].shift(1,fill_value=vpoc_at_930)    

In [11]:
df.to_csv('spy_engineered_v3.csv')